<h1><center><b>Laboratorio N° 02 - Utilizando Operadores de Conjuntos de Datos</b></center></h1>

<h2><b>Objetivo:</b></h2>
* Aplicar los conocimientos asimilados sobre operadores de conjuntos en SQL Server

<h2><b>Ejercicio 1: Operador UNION</b></h2>

1.1. Conectar con la BD TSQL

In [1]:
USE TSQL;
GO

Commands completed successfully.

Total execution time: 00:00:00

1.2. Escribir una sentencia SELECT que retorne las columnas productid y productname desde la tabla Production.Products. Filtrar los resultados  para incluir solo los productos que tienen la columna categoryid valor 4.

In [2]:
SELECT productid, productname
FROM Production.Products
WHERE categoryid = 4;

(10 rows affected)

Total execution time: 00:00:00.004

productid,productname
11,Product QMVUN
12,Product OSFNS
31,Product XWOXC
32,Product NUNAW
33,Product ASTMN
59,Product UKXRI
60,Product WHBYK
69,Product COAXA
71,Product MYMOI
72,Product GEEOO


1.3. Escribir la sentencia SELECT para retornar las columnas productid y productname desde la tabla Production.Products. Filtrar los resultados para incluir solo los productos que tienen un monoto total de ventas mayor a $50,000. Para el monto total de ventas, se necesitara la consultar la tabla Sales.OrderDetails y agregar todos los valores (qty * unitprice) de detalle o linea de orden por cada producto.

In [3]:
SELECT p.productid, p.productname
FROM Production.Products p
JOIN Sales.OrderDetails od ON p.productid = od.productid
GROUP BY p.productid, p.productname
HAVING SUM(od.qty * od.unitprice) > 50000;

(4 rows affected)

Total execution time: 00:00:00.008

productid,productname
29,Product VJXYN
38,Product QDOMO
59,Product UKXRI
60,Product WHBYK


1.4. Escribir una consulta SELECT para traer las columnas custid y contactname de la tabla Sales.Customers. Monstrar el top 10 de clientes por monto de ventas para el periodo January 2008 y mostrar el top 10 de clientes por monto de ventas para el periodo February 2008. (Ojo: escribir dos sentencias SELECT, enlazar Sales.Customers y Sales.OrderValues, y utilizar el apropiado operador de conjunto).

In [9]:
SELECT TOP 10 c.custid, c.contactname
FROM Sales.Customers c
JOIN Sales.OrderValues ov ON c.custid = ov.custid
WHERE ov.orderdate >= '2008-01-01' AND ov.orderdate < '2008-02-01'
GROUP BY c.custid, c.contactname;

(3 rows affected)

Total execution time: 00:00:00.006

custid,contactname
42,"Steiner, Dominik"
55,"Egelund-Muller, Anja"
88,"Li, Yan"


In [10]:
SELECT TOP 10 c.custid, c.contactname
FROM Sales.Customers c
JOIN Sales.OrderValues ov ON c.custid = ov.custid
WHERE ov.orderdate >= '2008-02-01' AND ov.orderdate < '2008-03-01'
GROUP BY c.custid, c.contactname;

(2 rows affected)

Total execution time: 00:00:00.007

custid,contactname
47,"Lupu, Cornel"
66,"Voss, Florian"


<h2><b>Ejercicio 2: Operador EXCEPT y INTERSECT</b></h2>

2.1. Escribir una sentencia SELECT para traer la columna custid de la tabla Sales.Orders table. Filtrar los resultados para incluir solo los clientes quierenes compraron mas de 20 diferentes productos (basada en la columna productid de la tabla Sales.OrderDetails).

In [11]:
SELECT o.custid
FROM Sales.Orders o
JOIN Sales.OrderDetails od ON o.orderid = od.orderid
GROUP BY o.custid
HAVING COUNT(DISTINCT od.productid) > 20;

(33 rows affected)

Total execution time: 00:00:00.013

custid
4
5
7
9
10
20
24
25
30
34


2.2. Escrbir una sentencia SELECT para traer la columna custid desd la tabla the Sales.Orders. Filtrar los resultados para incluir solo los clientes del pais USA y excluir todos los clientes del resultado previo (tarea 1). (Ojo: usar el operador EXCEPT la consulta anterior).

In [12]:
SELECT o.custid
FROM Sales.Orders o
WHERE o.custid IN (
    SELECT c.custid
    FROM Sales.Customers c
    WHERE c.country = 'USA'
)
EXCEPT
SELECT o.custid
FROM Sales.Orders o
JOIN Sales.OrderDetails od ON o.orderid = od.orderid
GROUP BY o.custid
HAVING COUNT(DISTINCT od.productid) > 20;

(10 rows affected)

Total execution time: 00:00:00.016

custid
32
36
43
45
48
55
75
77
78
82


2.3. Escribir una sentencia SELECT para traer la columna custid de la tabla Sales.Orders. Filtrar solo los clientes quienes tienen un valor de total de ventas mayor a $10,000. Calcular el valor de ventas utilizando las columnas qty y unitprice de la tabla Sales.OrderDetails.

In [13]:
SELECT o.custid
FROM Sales.Orders o
JOIN Sales.OrderDetails od ON o.orderid = od.orderid
GROUP BY o.custid
HAVING SUM(od.qty * od.unitprice) > 10000;

(41 rows affected)

Total execution time: 00:00:00.017

custid
23
46
75
9
89
72
32
35
86
63


<h2><b>Ejercicio 3: Utilizando el Operador APPLY</b></h2>

3.1. Escribir una sentencia SELECT para traer las columnas productid y productname de la tabla Production.Products. En adición, para cada producto, traer las ultimas dos filas de la tabla Sales.OrderDetails basado en el número orderid.

In [14]:
SELECT p.productid, p.productname, od.orderid, od.qty, od.unitprice
FROM Production.Products p
LEFT JOIN (
    SELECT od.*
    FROM Sales.OrderDetails od
    WHERE od.orderid IN (
        SELECT DISTINCT orderid
        FROM Sales.OrderDetails
        ORDER BY orderid DESC
        LIMIT 2
    )
) od ON p.productid = od.productid;

: Msg 102, Level 15, State 1, Line 10
Sintaxis incorrecta cerca de 'LIMIT'.

Total execution time: 00:00:00.003

3.2. Utilizar el operador CROSS APPLY y una subconsulta correlacionada. Ordenar el resultado por la columna productid.

3.3. Ejecutar la sentencia siguiente para crear la función fnGetTop3ProductsForCustomer.

In [1]:
DROP FUNCTION IF EXISTS dbo.fnGetTop3ProductsForCustomer; 
GO 
CREATE FUNCTION dbo.fnGetTop3ProductsForCustomer (@custid AS INT) 
RETURNS TABLE AS 
RETURN 
SELECT TOP(3) d.productid, p.productname, SUM(d.qty * d.unitprice) AS totalsalesamount 
FROM Sales.Orders AS o 
INNER JOIN Sales.OrderDetails AS d ON d.orderid = o.orderid 
INNER JOIN Production.Products AS p ON p.productid = d.productid 
WHERE custid = @custid 
GROUP BY d.productid, p.productname 
ORDER BY totalsalesamount DESC;

3.4. Escribir una sentencia SELECT statement para traer las columnas custid y contactname de la tabla Sales.Customers. Utilizar el operador CROSS APPLY con la función dbo.fnGetTop3ProductsForCustomer para traer las columnas productid, productname y totalsalesamount por cada cliente.